# Shane's Swing Trading Strategy

* Stocks have support levels
    * e.g. MSFT hits recent low of $400 on May 1st, then hits a similar low of $390 on Aug 5th
    * Stocks reaching support levels is a buying opportunity
* However, good stocks trend up
    * This means the support level of good stocks rise over time
    * Thus it's difficult to rely on historical price data to determine support levels
* To adjust for this, we calculate support levels not by historical price but rather historical difference between price and 200 SMA
    * 200 SMA is a good indicator for the general direction of a stock
    * The difference between the price and the 200 SMA then measures the residual of the price from its expected price

In [ ]:
import alpaca_trade_api as alpaca
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from dotenv import load_dotenv
import os
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import time
import datetime as dt

In [ ]:
# initialize API from API keys in .env
load_dotenv()
api_key = os.environ['APCA-API-KEY-ID']
api_secret_key = os.environ['APCA-API-SECRET-KEY']
api_base_url = 'https://paper-api.alpaca.markets'
api = alpaca.REST(api_key, api_secret_key, api_base_url)
account = api.get_account()
trading_client = TradingClient(api_key, api_secret_key, paper=True)

In [ ]:
def market_buy_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.BUY,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)
    
def market_sell_order(ticker, quantity):
    time.sleep(0.2)
    market_order_data = MarketOrderRequest(
                        symbol=ticker,
                        qty=quantity,
                        side=OrderSide.SELL,
                        time_in_force=TimeInForce.DAY
                        )
    market_order = trading_client.submit_order(order_data=market_order_data)
    print(market_order)

In [ ]:
def get_SP500():
    return pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

def add_business_days(start_date: dt.date, n: int) -> dt.date:
    weeks, extra_days = divmod(n, 5)
    result_date = start_date + dt.timedelta(weeks=weeks)

    if extra_days != 0:
        start_weekday = result_date.weekday()
        
        if start_weekday + extra_days > 4:
            extra_days += 2
        result_date += dt.timedelta(days=extra_days)
    return result_date

def bars_last_n_days(ticker, days):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(days=days)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
    return bars

def bars_last_n_weeks(ticker, weeks):
    now = dt.datetime.now(dt.timezone.utc)
    todays_date = now.strftime("%Y-%m-%d")
    start_date = (now - dt.timedelta(weeks=weeks)).strftime("%Y-%m-%d")
    fh_date_time = start_date + "T14:30:00Z"
    bars = api.get_bars(ticker, alpaca.TimeFrame.Day, fh_date_time, adjustment='split').df
    bars = bars.reset_index()
    bars = bars.rename(columns={'timestamp': 'date'})
    bars['date'] = pd.to_datetime(bars['date']).dt.date
    bars = bars.set_index('date')
    return bars

def add_SMA(bars, days):
    bars[f"{days} day SMA"] = ta.SMA(bars['close'], timeperiod=days)
    bars[f"close - {days} day SMA"] = bars['close'] - bars[f"{days} day SMA"]
    return bars[bars[f"{days} day SMA"].notna()]

def SMA_slope(bars, sma_days):
    time_period = 5
    return (bars.iloc[-1][f"{sma_days} day SMA"] - bars.iloc[-1 - time_period][f"{sma_days} day SMA"]) / time_period

def get_troughs(bars, n_troughs, radius):
    trough_dates = []
    sorted_bars = bars.sort_values('close - 200 day SMA')
    for i in range(len(sorted_bars)):
        close_prox = False
        row = sorted_bars.iloc[i]
        date = row.name
        for tdate in trough_dates:
            lower_bound = add_business_days(tdate, -radius)
            upper_bound = add_business_days(tdate, radius)
            if date >= lower_bound and date <= upper_bound:
                close_prox = True
                break
        if close_prox:
            continue
        trough_dates.append(date)
        if len(trough_dates) >= n_troughs:
            break
    return sorted_bars[sorted_bars.index.isin(trough_dates)].sort_index()

In [ ]:
bars = bars_last_n_weeks('META', 85)
bars = add_SMA(bars, 200)
print(SMA_slope(bars, 200))
display(bars)
bars.sort_values('close - 200 day SMA').head(20)

In [ ]:
plt.plot(bars['close - 200 day SMA'])
plt.show()
plt.clf()

In [ ]:
troughs = get_troughs(bars, 7, 5)
troughs_range = troughs['close - 200 day SMA'].max() - troughs['close - 200 day SMA'].min()
troughs_std = troughs['close - 200 day SMA'].std()
troughs_mean = troughs['close - 200 day SMA'].mean()
print(troughs_range)
print(troughs_std)
print(troughs_mean)
troughs

In [ ]:
print(api.get_latest_quote('META'))
print(api.get_latest_trade('META'))

In [ ]:
bars = api.get_bars("SPY", alpaca.TimeFrame.Day, "2021-06-01", "2024-11-18").df

bars


In [ ]:
bars['30_Day_SMA'] = ta.SMA(bars['close'], timeperiod=30)


plt.plot(bars['30_Day_SMA'], label='30 SMA')
plt.plot(bars['close'], label = 'close')
plt.legend()
plt.show()
plt.clf()


In [ ]:
date = '2024-11-15'
market_open_datetime = date + "T14:30:00Z"
market_close_datetime = date + "T21:00:00Z"
bars2 = api.get_bars('SPY', alpaca.TimeFrame.Minute, market_open_datetime, market_close_datetime).df 

print(bars2.to_string())
bars2

In [ ]:
api.get_latest_quote('SPY')

In [ ]:
tickers = pd.read_csv('trading_tickers.csv')

tickers

In [ ]:
# POTENTIAL STRATEGY
# here's an example of actual support levels
#   MSFT hits recent low of $400 on May 1st, then hits a similar low of $390 on Aug 5th
# however, good stocks tend to trend up. we can measure this with a 200 SMA
# we can then calculate adjusted support levels based on the difference from the 200 SMA, which acts as the expected trajectory
# we calculate the 200 SMA for the last 200 days, which requires 400 days of data

In [ ]:
SP500 = get_SP500()
SP500